In [1]:
import numpy as np
import functools

types = {'A':1, 'B':2, 'C':3, 'D':4} #dont use 0s so I dont fuck up a bool or smthing
costs = np.array((0, 1, 10, 100, 1000)) #(costs[type] = cost of type)
with open("input.txt") as f:
    data = "".join(['0' for j in range(7)] + [str(types[i]) for i in \
            np.array([list(f"{x[3:10:2]}") for x in f.read().split('\n')[2:4]]).T.flatten()])
data

'000000043114232'

In [2]:
def print_indexes():
    out = f"#################\n#01 .2 .3 .4 .56#\n### 7# 9#11#13###\n  # 8#10#12#14#\n  #############"
    return out
def state_str(state):
    _state = state.replace('0','.').replace('1','A').replace('2','B').replace('3','C').replace('4','D')
    hallway = "#" + _state[0] + ".".join(list(_state[1:6])) + _state[6] + "#"
    rooms = "###" + "#".join(_state[7::2]) + "###\n  #" + "#".join(_state[8::2]) +"#"
    out = f"#############\n{hallway}\n{rooms}\n  #########"
    return out


In [3]:
t_inds = lambda x: (x-5)//2 + 0.5 #index at which room is entered      

def cost_to_room(board, i, t):
    '''cost from hallway index i to room at index t. None if the path is impassable.
        Does not check if i -> t is legal, nor for room occupation'''
    ti = t_inds(t)
    moved = 0
    _i = i #find out how many other room enterances are passed
    if _i == 0:
        _i = 1
    if _i == 6:
        _i = 5

    if i < ti: #has to move to the right
        ti = int(ti+0.5) # round down
        if int('0' + board[i+1:ti]) > 0: #value other than 0 in path
            return None
        moved += ti-i  # hallway spaces
        moved += ti-_i # room enterances
    else: # ti > i
        ti = int(ti)
        if int('0' + board[ti+1:i]) > 0: 
            return None
        moved += i-ti
        moved += _i-ti
    moved += (t+1)%2 # move to bottom of room
    return moved

def cost_to_hw(board, i, t):
    moved = (i+1)%2 # move into room enterance
    _i = t_inds(i) #where is room enterance
    _t = t
    if _t == 0:
        _t = 1
    if _t == 6:
        _t = 5
    if t > _i: #moves right
        _i = int(_i)
        if int('0' + board[_i+1:t+1]) > 0:
            return None
        moved += _t - _i
        moved +=  t - _i
    else: #moves left
        _i = int(_i+0.5)
        if int('0' + board[t:_i]) > 0:
            return None
        moved += _i -  t
        moved += _i - _t
    return moved
cost_to_hw('100000021023344',7,3)

4

In [4]:
@functools.lru_cache(maxsize=0)
def get_moves(state):
    moves = []
    empty_hw = []
    for i in range(7): #moves from hallway
        val = int(state[i])
        if val == 0:
            empty_hw.append(i)
            continue
        room = [int(x) for x in state[5+2*val:7+2*val]]
        target = -1
        if room[0] != 0:
            continue
        if room[1] == 0:
            target = 6+2*val
        elif room[1] != val:
            continue
        else:
            target = 5+2*val
        
        cost = cost_to_room(state, i, target)
        if cost is None:
            continue
        cost *= costs[val]
        moves.append((i, target, cost))
    
    #moves to hallway
    for i in range(7, 15):
        val = int(state[i])
        if val == 0:
            continue
        #make sure no moves out of correct room
        if i == 6+2*val:
            continue
        if (i == 5+2*val) and (int(state[6+2*val]) == val):
            continue
        if i%2 == 0 and int(state[i-1]) != 0:
            continue
        for t in empty_hw:
            cost = cost_to_hw(state, i, t)
            if cost is None:
                continue
            cost *= costs[val]
            moves.append((i, t, cost))
    return moves

def do_move(board, i, t): #simple function to swap two indexes in a string, order doesnt matter
    im, il = sorted((i,t))
    return board[:im] + board[il] + board[im+1:il] + board[im] + board[il+1:]

do_move("123456", 5, 2)

'126453'

In [5]:
@functools.lru_cache(maxsize=0)
def heuristic(state): #cost left
    c = 0
    for i in range(7): #everything in a hallway
        val = int(state[i])
        if val == 0:
            continue
        c += (2 + abs(i - t_inds(6 + 2*val))) * costs[val]
    for i in range(7,15): #starting in rooms
        val = int(state[i])
        if val == 0:
            continue
        if abs(5.5+2*val - i) < 1:
            continue #right spot
        c += ((i+1)%2 + 3 + abs(i - t_inds(6+2*val))) * costs[val]
    return c

def is_done(state):
    return state == '000000011223344'

@functools.lru_cache(maxsize=None)            
def dijkstra(initial):
    if is_done(initial):
        return 0, [initial]
    best = float('inf')
    best_path = []
    moves = get_moves(initial)
    nxt_boards = [(do_move(initial, move[0], move[1]), move[2]) for move in moves]
    for nxt in nxt_boards:
        cost_to_end, _best_path = dijkstra(nxt[0])
        cost_to_end += nxt[1]
        if cost_to_end < best:
            best = cost_to_end
            best_path = _best_path
    
    return best, [initial] + best_path
cst, path = dijkstra(data)
print(f"Final cost: {cst}")
#for p in path:
#    print(state_str(p), end='\n\n')


Final cost: 14546


Part Two:

In [6]:
data2 = np.array(list(data[7:])).reshape(4,2).T
data2 = "".join(['0' for _ in range(7)]) + "".join(np.vstack((data2[0,:], np.matrix("4 3 2 1 ; 4 2 1 3"), data2[1,:])).T.flatten().tolist()[0])
data2

'00000004443132142123132'

In [7]:
def print_indexes2():
    out = f"#################\n#01 .2 .3 .4 .56#\n### 7#11#15#19###\n  # 8#12#16#20#\n  # 9#13#17#21#\n  #10#14#18#22#\n  #############"
    return out
def state_str2(state):
    _state = state.replace('0','.').replace('1','A').replace('2','B').replace('3','C').replace('4','D')
    hallway = "#" + _state[0] + ".".join(list(_state[1:6])) + _state[6] + "#"
    rooms = "###" + "#".join(_state[7::4]) + "###\n  #" + "#".join(_state[8::4]) + "#\n  #" +"#".join(_state[9::4]) + "#\n  #" + "#".join(_state[10::4]) + "#"
    out = f"#############\n{hallway}\n{rooms}\n  #########"
    return out
print(print_indexes2())
print(state_str2(data2))

#################
#01 .2 .3 .4 .56#
### 7#11#15#19###
  # 8#12#16#20#
  # 9#13#17#21#
  #10#14#18#22#
  #############
#############
#...........#
###D#A#D#C###
  #D#C#B#A#
  #D#B#A#C#
  #C#A#B#B#
  #########


In [ ]:
t_inds2 = lambda x: (x-3)//4 + 0.5 #index at which room is entered      

def cost_to_room2(board, i, t):
    '''cost from hallway index i to room at index t. None if the path is impassable.
        Does not check if i -> t is legal, nor for room occupation'''
    ti = t_inds2(t)
    moved = 0
    _i = i #find out how many other room enterances are passed
    if _i == 0:
        _i = 1
    if _i == 6:
        _i = 5

    if i < ti: #has to move to the right
        ti = int(ti+0.5) # round down
        if int('0' + board[i+1:ti]) > 0: #value other than 0 in path
            return None
        moved += ti-i  # hallway spaces
        moved += ti-_i # room enterances
    else: # ti > i
        ti = int(ti)
        if int('0' + board[ti+1:i]) > 0: 
            return None
        moved += i-ti
        moved += _i-ti
    moved += (t+3)%4 # move to bottom of room
    return moved

def cost_to_hw2(board, i, t):
    moved = (i+1)%4 # move into room enterance
    _i = t_inds2(i) #where is room enterance
    _t = t
    if _t == 0:
        _t = 1
    if _t == 6:
        _t = 5
    if t > _i: #moves right
        _i = int(_i)
        if int('0' + board[_i+1:t+1]) > 0:
            return None
        moved += _t - _i
        moved +=  t - _i
    else: #moves left
        _i = int(_i+0.5)
        if int('0' + board[t:_i]) > 0:
            return None
        moved += _i -  t
        moved += _i - _t
    return moved


In [9]:
def get_moves2(state):
    moves = []
    empty_hw = []
    for i in range(7): #moves from hallway
        val = int(state[i])
        if val == 0:
            empty_hw.append(i)
            continue
        target = -1
        for t in range(3+4*val, 7+4*val):
            if int(state[t]) == 0:
                target = t
            elif int(state[t]) != val:
                target = -1
        if target == -1:
            continue
        cost = cost_to_room2(state, i, target)
        if cost is None:
            continue
        cost *= costs[val]
        moves.append((i, target, cost))
    
    #moves to hallway
    for i in range(7, len(state)):
        val = int(state[i])
        if val == 0:
            continue
        #make sure no moves out of correct room
        flag = False
        if val == (i-3)//4: #right room
            flag = True
            for j in range(i, 4*val+7):
                if int(state[j]) != val:
                    flag = False
                    break
        if flag:
            continue
        flag = False
        for j in range(i-(i+1)%4, i): #check everyone above
            if state[j] !='0':
                flag = True
                break
        if flag:
            continue
        for t in empty_hw:
            cost = cost_to_hw2(state, i, t)
            if cost is None:
                continue
            cost *= costs[val]
            moves.append((i, t, cost))
    return moves


In [10]:
@functools.lru_cache(maxsize=0)
def heuristic2(state): #cost left
    c = 0
    for i in range(7): #everything in a hallway
        val = int(state[i])
        if val == 0:
            continue
        c += (2 + abs(i - t_inds2(3+4*val))) * costs[val]
    for i in range(7,15): #starting in rooms
        val = int(state[i])
        if val == 0:
            continue
        if abs(5.5+2*val - i) < 1:
            continue #right spot
        c += ((i+1)%4 + 3 + abs(i - t_inds(3+4*val))) * costs[val]
    return c

def is_done2(state):
    return state == '00000001111222233334444'

@functools.lru_cache(maxsize=None)            
def dijkstra2(initial):
    if is_done2(initial):
        return 0, [initial]
    best = float('inf')
    best_path = []
    moves = get_moves2(initial)
    nxt_boards = [(do_move(initial, move[0], move[1]), move[2]) for move in moves]
    for nxt in nxt_boards:
        cost_to_end, _best_path = dijkstra2(nxt[0])
        cost_to_end += nxt[1]
        if cost_to_end < best:
            best = cost_to_end
            best_path = _best_path
    
    return best, [initial] + best_path
cst, path = dijkstra2(data2)
print(f"Final cost: {cst}")
for p in path:
    print(state_str2(p), end='\n\n')


Final cost: 42308
#############
#...........#
###D#A#D#C###
  #D#C#B#A#
  #D#B#A#C#
  #C#A#B#B#
  #########

#############
#C..........#
###D#A#D#.###
  #D#C#B#A#
  #D#B#A#C#
  #C#A#B#B#
  #########

#############
#CA.........#
###D#A#D#.###
  #D#C#B#.#
  #D#B#A#C#
  #C#A#B#B#
  #########

#############
#CA.D.......#
###.#A#D#.###
  #D#C#B#.#
  #D#B#A#C#
  #C#A#B#B#
  #########

#############
#CA.D...D...#
###.#A#.#.###
  #D#C#B#.#
  #D#B#A#C#
  #C#A#B#B#
  #########

#############
#CA.D...D..C#
###.#A#.#.###
  #D#C#B#.#
  #D#B#A#.#
  #C#A#B#B#
  #########

#############
#CA.D...D.BC#
###.#A#.#.###
  #D#C#B#.#
  #D#B#A#.#
  #C#A#B#.#
  #########

#############
#CA.D.....BC#
###.#A#.#.###
  #D#C#B#.#
  #D#B#A#.#
  #C#A#B#D#
  #########

#############
#CA.......BC#
###.#A#.#.###
  #D#C#B#.#
  #D#B#A#D#
  #C#A#B#D#
  #########

#############
#CA.D.....BC#
###.#A#.#.###
  #.#C#B#.#
  #D#B#A#D#
  #C#A#B#D#
  #########

#############
#CA.......BC#
###.#A#.#.###
  #.#C#B#D#
  #D#B#A#D#
  #C#A